In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from models.Resnet50 import resnet50
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [2]:
img_size = 64
batch_size = 20

In [3]:
# 先把根目錄用到food11
import os
# load dataset
dataset_dir = './food11'
# create new dir
try:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')
    for i in range(11):
        os.mkdir(f'train/{i:02d}')
        os.mkdir(f'valid/{i:02d}')
except:
    pass

In [4]:
# create new file structure for dataset
for sub_dir in ['training', 'validation', 'test']:
    target_dir = os.path.join(dataset_dir, sub_dir)
    img_list = os.listdir(target_dir)
    for img_name in img_list:
        category, *r = img_name.split('_')
        img_path = os.path.join(target_dir, img_name)
        if len(r) == 0:
            new_fn = f'{sub_dir}/{img_name}'
        else:
            category = int(category)
            new_fn = f'{sub_dir[:5]}/{category:02d}/{img_name}'
    # resize image
        os.rename(img_path, new_fn)

In [5]:
# Dataset preprocessing
import tensorflow as tf
valid_data_pro = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_data_aug = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True) #image augmentation
train_data = train_data_aug.flow_from_directory(
        'train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="sparse")
valid_data = valid_data_pro.flow_from_directory(
        'valid',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="sparse")
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    'test',
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    image_size=(img_size, img_size),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)
test_set = test_set.map(lambda x: (tf.divide(x, 255)))

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 files belonging to 1 classes.


In [6]:
model = resnet50()
model.summary()
plot_model(model, to_file='ResNet50.png', show_shapes=True)
##調整學習率，採用學習率隨epoch數上升呈遞減的狀態，避免一開始學習過慢，之後overshooting的情形
starter_learning_rate = 1e-2
end_learning_rate = 1e-3
decay_steps = 30000
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(starter_learning_rate, decay_steps, end_learning_rate, power=0.6, cycle=True)
model_dir=r'.\models\Resnet50'
os.makedirs(model_dir)
log_dir = os.path.join('exam','model')
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp=keras.callbacks.ModelCheckpoint(model_dir + '/Best_exam_model.h5', monitor='val_sparse_categorical_accuracy',save_best_only=True,mode='max')
model.compile(tf.keras.optimizers.SGD(learning_rate=learning_rate_fn),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])
##根據林益帆同學發現的結果，改用遷移學習，準確率可顯著提升
history_1=model.fit(train_data,epochs=30,validation_data=valid_data,callbacks=[model_cbk,model_mckp])

Model: "Resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
___________________________________________________________________________________________

Epoch 1/30
494/494 [==============================] - 52s 88ms/step - loss: 11.3366 - sparse_categorical_accuracy: 0.1260 - val_loss: 5.9747 - val_sparse_categorical_accuracy: 0.1519
Epoch 2/30
494/494 [==============================] - 36s 74ms/step - loss: 6.4254 - sparse_categorical_accuracy: 0.1527 - val_loss: 5.4537 - val_sparse_categorical_accuracy: 0.2155
Epoch 3/30
494/494 [==============================] - 38s 76ms/step - loss: 5.8126 - sparse_categorical_accuracy: 0.1967 - val_loss: 5.3488 - val_sparse_categorical_accuracy: 0.2536
Epoch 4/30
494/494 [==============================] - 36s 74ms/step - loss: 5.5375 - sparse_categorical_accuracy: 0.2256 - val_loss: 5.3633 - val_sparse_categorical_accuracy: 0.1936
Epoch 5/30
494/494 [==============================] - 36s 73ms/step - loss: 5.3297 - sparse_categorical_accuracy: 0.2648 - val_loss: 5.1805 - val_sparse_categorical_accuracy: 0.2458
Epoch 6/30
494/494 [==============================] - 36s 74ms/step - loss: 5.2007 - spar

In [9]:
model.load_weights('models\Resnet50\Best_exam_model.h5')
answer = model.predict(test_set)
df = pd.DataFrame()
df["Id"] = ["0"*(4-len(str(i)))+str(i) for i in range(1,np.shape(answer)[0]+1)]
def get_class(i):
    top_1_index = np.argsort(answer[i])[10]
    return top_1_index 
df["Category"] = [get_class(i) for i in range(0,np.shape(answer)[0])]
df.to_csv("submission.csv",index = False)